### 1. 패키지 설치

### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [1]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"로그인 상태입니다. 사용자: {user_info['name']}")
except Exception as e:
    print("로그인되지 않았거나 토큰이 유효하지 않습니다.")
    print(e)

/home/chae/ch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


로그인 상태입니다. 사용자: chaeeee


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.schema import Document

# 텍스트 분할 설정
text_splitter = CharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=128  # 오버랩 설정
)

# 단일 DOCX 파일 로드
file_path = "./dataset2.docx"  # 파일 경로를 이곳에 입력하세요
loader = Docx2txtLoader(file_path)
raw_text = loader.load()[0].page_content  # DOCX 파일의 전체 텍스트 가져오기
print("raw_text 개수: ", len(raw_text))
      
# 작품명을 기준으로 텍스트 분리
def split_artwork_documents(doc_text):
    artworks = doc_text.split("\n\n작품명:")  # 작품을 구분
    documents = []

    for artwork in artworks:
        if artwork.strip():  # 빈 텍스트 제외
            # "작품명:" 추가로 일관성 유지
            doc_content = "작품명:" + artwork if not artwork.startswith("작품명:") else artwork

            # 메타데이터 초기화
            metadata = {}
            lines = doc_content.split("\n")  # 텍스트 줄 단위로 나누기

            # 메타데이터 추출
            for line in lines:
                if line.startswith("작품명:"):
                    metadata["작품명"] = line.replace("작품명:", "").strip()
                elif line.startswith("작가:"):
                    metadata["작가"] = line.replace("작가:", "").strip()
                elif line.startswith("제작 연도:"):
                    metadata["제작 연도"] = line.replace("제작 연도:", "").strip()
                elif line.startswith("카테고리:"):
                    metadata["카테고리"] = line.replace("카테고리:", "").strip()

            # Document 객체 생성
            documents.append(Document(
                page_content=doc_content.strip(),
                metadata=metadata
            ))

    return documents



# 작품별 Document 생성
documents = split_artwork_documents(raw_text)

# 생성된 작품별 Document에 대해 추가 청크 분할
chunked_documents = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)
    for chunk in chunks:
        # 청크에 원래 Document의 메타데이터 유지
        chunked_documents.append(Document(
            page_content=chunk,
            metadata=doc.metadata  # 원본 메타데이터 복사
        ))

print(f"총 문서 수: {len(chunked_documents)}")

raw_text 개수:  8042447
총 문서 수: 15195


In [3]:
# 첫 2개 문서만 출력해 확인
for i, doc in enumerate(chunked_documents[:2], 1):
    print(f"Document {i}:")
    print("메타데이터:", doc.metadata)
    print(doc.page_content)
    print("-" * 50)

Document 1:
메타데이터: {'작품명': '작가사진 / 作家寫眞 / Selfportrait', '작가': '한기석 / HAN Kisuk', '제작 연도': '1960', '카테고리': '사진'}
작품명: 작가사진 / 作家寫眞 / Selfportrait

작가: 한기석 / HAN Kisuk

작품 번호: 1

제작 연도: 1960

크기: 41×51

재료: 종이에 젤라틴실버프린트

카테고리: 사진

작품 설명: ‘농(Nong)’이라는 이름으로 미국에서 널리 알려진 한농(韓農) 한기석(1930-2011)은 국내 활동이 그리 많지 않아서 한국 화단에서는 생소한 이름이다. 그가 최초로 한국 화단에 등장한 것은 1971년 11월 신세계 화랑에서 개최한《Nong 展》이후이다. 그는 농(Nong)을 구름 위의 시선(詩仙) 혹은 주선(酒仙)같은 존재로 비유해서 미국에서 자신의 이름으로 쓰고 있다.그의 작품은 전반적으로 자신의 철학적 이미지를 조형화시킨 추상 회화 계통이다. 일종의 형이상학적인 회화 혹은 초현실적인 환상세계라고도 할 수 있는 그의 작품은 양식적인 면에서 주로 구상적인 형태를 취한다.한기석의 <작가사진>(1960)은 본인의 얼굴을 찍은 것으로, 사진 속에서 작가는 자신의 작품을 배경으로 화면의 우측을 주시하고 있다.
--------------------------------------------------
Document 2:
메타데이터: {'작품명': '팔괘호 / 八卦壺 / Palgwae Vase', '작가': '한기석 / HAN Kisuk', '제작 연도': '1960', '카테고리': '회화 II'}
작품명: 팔괘호 / 八卦壺 / Palgwae Vase

작가: 한기석 / HAN Kisuk

작품 번호: 2

제작 연도: 1960

크기: 250×127

재료: 캔버스, 종이에 유화 물감

카테고리: 회화 II

작품 설명: 한농(韓農) 한기석(1930-2011)은 표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는 

In [4]:
documents[11000]

Document(metadata={'작품명': '직각 / N/A / Right Angle', '작가': '김차섭 / KIM Tchahsup', '제작 연도': '1981', '카테고리': '판화'}, page_content='작품명: 직각 / N/A / Right Angle\n\n작가: 김차섭 / KIM Tchahsup\n\n작품 번호: 11001\n\n제작 연도: 1981\n\n크기: 57×76.5\n\n재료: 종이에 에칭\n\n카테고리: 판화\n\n작품 설명: 김차섭(金次燮, 1942-2022)은 일본 야마구치현(山口県)에서 태어났으며, 광복 즈음에 한국으로 건너와 1963년 서울대학교 회화과에 입학했다. 1960년대 후반에 《회화 ‘68》전과 《AG》전에 참여했으며, 1967년 제5회 파리비엔날레, 1970년 제7회 도쿄 국제판화비엔날레, 1971년 제11회 상파울루비엔날레 등에 참가하며 국제적으로도 활발히 활동하였다. 김차섭은 록펠러 재단 장학금을 받고 1975년에 미국으로 건너가서 뉴욕 프랫 인스티튜트(Pratt Institute) 대학원에서 에칭을 공부했다. 1990년에 귀국한 후 춘천의 한 폐교에 작업실을 마련하여 뉴욕과 한국을 오가며 작업했다. 작가는 1975년에 포드 재단 프랫 인스티튜트 스튜디오 미술상을 받고, 2002년 제14회 이중섭미술상, 2008년 제9회 이인성미술상 등을 수상했다. 김차섭은 1970년대까지 스크린 판화를 다수 제작했으나, 미국으로 건너간 후에는 에칭 기법을 활용한 판화작업을 했다. 김차섭의 에칭 판화는 기법뿐 아니라 작가가 그 당시 추구했던 기하학적 개념을 처음으로 시도한 것이었다.<직각>은 김차섭이 1976년 미국으로 가기 전부터 부단히 고민했던 ‘삼각형’에 대한 관념이 드러나 있다. 삼각형의 한 꼭짓점이 뚫려 있고 이를 가리키는 듯이 제스쳐를 한 손이 그려져 있으며, 열려 있는 직각 부분을 손 뒤의 예리한 각이 짚고 있다. 긴장감 있으면서 완벽한 도형으로 인식되는 삼각형의 직각 부분을 뚫은 것은 서구 합리주의에 대

In [5]:
# 특정 Document 객체의 텍스트 길이 확인
len(documents[11000].page_content)

970

In [6]:
# 800자를 초과하는 Document 개수 세기
over_800_count = sum(1 for doc in documents if len(doc.page_content) > 800)

# 결과 출력
print(f"800자를 초과하는 Document 개수: {over_800_count}")


800자를 초과하는 Document 개수: 4166


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

# 올바른 Hugging Face 모델을 사용한 임베딩 생성
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')

# 확인
print("HuggingFaceEmbeddings initialized successfully!")


/tmp/ipykernel_828/746791366.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')


HuggingFaceEmbeddings initialized successfully!


In [8]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# 3. Chroma 데이터베이스 초기화 및 문서 추가
collection_name = 'chroma_art'

database = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name=collection_name,
    persist_directory='./chroma_1218'
)

print("Chroma database initialized successfully!")

Chroma database initialized successfully!


In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

database = Chroma(
    persist_directory="./chroma_1205",       # 기존 데이터베이스 경로
    embedding_function=embeddings,     # 동일한 임베딩 모델
    collection_name="chroma_art"            # 컬렉션 이름
)

print("기존 Chroma 데이터베이스가 성공적으로 로드되었습니다!")


/tmp/ipykernel_828/3561701732.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  database = Chroma(


기존 Chroma 데이터베이스가 성공적으로 로드되었습니다!


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [6]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 7/7 [00:23<00:00,  3.37s/it]


In [7]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipykernel_828/1315975019.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [8]:
import re

class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        # <assistant> 이후의 텍스트만 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        if match:
            extracted_text = match.group(1).strip()
            # 마크다운 코드 블록으로 출력 포맷
            return f"```markdown\n{extracted_text}\n```"
        else:
            # <assistant> 태그가 없는 경우 원래 출력 반환
            return f"```markdown\n{llm_output.strip()}\n```"


In [9]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks. 
Answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with "The document does not contain this information." 
Provide detailed and comprehensive answers, always include the artwork ID, and ensure all answers are written in Korean.

<|context|>
{context}

<|history|>
{history}

<|user|>
Question: {question}

<|assistant|>
'''


# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [10]:
retriever = database.as_retriever(search_kwargs={"k": 5})

In [15]:
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers.string import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# 체인 구성 수정
chain = (
    RunnableMap({
        "context": retriever,  # 문서 검색기에서 context 가져오기
        "history": lambda _: memory.load_memory_variables({}).get("history", ""),  # 메모리에서 요약된 대화 가져오기
        "question": RunnablePassthrough()  # 입력된 질문 그대로 전달
    })
    | prompt  # 프롬프트에 context, history, question 전달
    | llm  # LLM 실행
    | MarkdownOutputParser()  # 출력 결과를 마크다운으로 포맷팅
)

In [20]:
query = "김기승 작가 작품 2개 추천해줘."

In [21]:
# 마지막에 원하는 형식으로 출력 필터링
response = chain.invoke(query)
print(response)

```markdown
김기승 작가의 작품 중 두 가지를 추천드리겠습니다:

1. **작품명:** 사삼공산비(예서) / 祀三公山碑(隸書) /  
   **작품 번호:** 1166  
   **설명:** 이 작품은 연미하고 날렵하며 여성적인 미감을 지닌 예서체로, 부드럽고 우아한 필체를 선호하는 사람들에게 적합한 서예 작품입니다.

2. **작품명:** 예서(조전비문 중에서) / 隸書(曺全碑文 中에서) /  
   **작품 번호:** 1192  
   **설명:** 이 작품은 연미하고 날렵한 예서체로, 부드럽고 아름다운 서체를 좋아하는 분들께 추천드립니다. 김기승 작가의 독특한 필세와 개성적인 감각이 잘 드러난 작품입니다.
```


In [14]:
import uuid

# 전역 변수로 세션 ID 저장
SESSION_ID = str(uuid.uuid4())

# 세션 ID 반환 함수
def get_session_id():
    return SESSION_ID


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory
from langchain.schema import HumanMessage, AIMessage
from langchain.llms import HuggingFacePipeline
import uuid

# 세션별 메모리 저장 딕셔너리
memory_store = {}

# 고정된 세션 ID를 생성
SESSION_ID = str(uuid.uuid4())


# Custom Prompt Template
summary_prompt = PromptTemplate(
    input_variables=["summary", "new_lines"],
    template="""
    Previous conversation summary:
    {summary}

    Newly added conversation:
    {new_lines}

    Please generate an updated summary:
    """
)


memory = ConversationSummaryMemory(
    llm=llm,
    prompt=summary_prompt,  # 위에서 정의한 프롬프트 전달
    return_messages=True
)

# save_context를 사용해 메모리 업데이트
memory.save_context(
    {"input": query},  # 사용자 입력
    {"output": response}    # 모델 응답
)
 

In [19]:
# Memory에서 요약 확인
summary = memory.load_memory_variables({})
history = summary.get("history", "요약이 없습니다.")

def extract_summary(history): 
    # 리스트의 각 메시지에서 content를 추출하고 하나의 문자열로 합침
    if isinstance(history, list):
        history = "\n".join(
            message.content for message in history if hasattr(message, "content")
        )
    
    # 정규식으로 "새로운 요약" 부분 추출
    match = re.search(r"Please generate an updated summary:\s*(.*)", history, re.DOTALL) 
    if match: 
        return match.group(1).strip() 
    return "요약이 없습니다."

clean_summary = extract_summary(history)

print("\n=== 저장된 대화 요약 ===")
print(clean_summary)



=== 저장된 대화 요약 ===
Here's the revised information about Lee Jung-seop's "Clover":

    * **Title:** Clover (also known as 토끼풀)
    * **Creation Year:** 1941
    * **Dimensions:** 9 x 14 inches
    * **Materials Used:** Pen on paper with watercolor paint
    * **Category:** Drawing

    **Description:** This drawing captures a simple yet profound moment of everyday joy through its depiction of clover blossoms and leaves arranged centrally within the composition. The interplay between bold pen lines infused with delicate watercolor washes imbues the artwork with vitality. Notably, contrasting hues create sharp clarity among individual elements like stems, foliage, and flower buds while adding dynamism to the overall piece thanks to blurred ink trails over painted areas. A personal touch lies in the artist’s fingerprint subtly embedded at the lower right corner, offering insight into his creative process during creation time. Reflecting deeper sentiments towards appreciating modest happin

In [51]:
# 대화 기록 확인
print("\n=== 대화 기록 ===")
for message in memory.chat_memory.messages:
    role = "User" if isinstance(message, HumanMessage) else "AI"
    print(f"{role}: {message.content}")


=== 대화 기록 ===
User: 이중섭 토끼풀 작품에 대해 알려줘
AI: ```markdown
이중섭의 "토끼풀" 시리즈는 작가의 표현주의적인 스타일과 자연에 대한 따뜻한 시선을 잘 보여주는 작품들입니다. 주요 작품들은 다음과 같습니다:

1. **작품명**: 토끼풀 / N/A / Clover
   - **작품 번호**: 10001
   - **제작 연도**: 1941
   - **크기**: 9×14 cm
   - **재료**: 종이에 펜, 수채 물감
   - **설명**: 이 작품은 토끼풀을 중심으로 화면 중앙에 크게 표현되어 있으며, 좌우로 꽃줄기와 뾰족한 풀잎이 사선으로 배치되어 조화롭게 구성되어 있습니다. 수채 물감과 펜의 사용으로 생동감이 넘치며, 작가의 일상적 행복을 갈망하는 마음이 담겨 있습니다.

2. **작품명**: 토끼풀과 새가 있는 바닷가 / N/A / Seashore with Clover and Birds
   - **작품 번호**: 10018
   - **제작 연도**: 1941
   - **크기**: 9×14 cm
   - **재료**: 종이에 펜, 수채 물감
   - **설명**: 이 작품은 바닷가 풍경과 함께 토끼풀이 등장하여 평화로운 분위기를 연출합니다. 토끼풀이 그림 전체에 역동성을 더하며, 자연과 조화로운 삶을 추구하는 작가의 마음을 반영하고 있습니다.

두 작품 모두 1941년에 제작되었으며, 이중섭의 독특한 표현 기법과 자연에 대한 애정을 잘 드러내고 있습니다. 이러한 작품들은 작가의 내면세계와 자연에 대한 깊은 애정을 잘 보여주는 예술적 표현입니다.
```


In [41]:
results = database.similarity_search(query, k=5)  # 상위 3개의 결과를 가져옴

# 결과 확인
for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(result["description"] if "description" in result else result)
    print("-" * 50)

Result 1:
page_content='작품명: 토끼풀 / N/A / Clover

작가: 이중섭 / LEE Jungseop

작품 번호: 10001

제작 연도: 1941

크기: 9×14

재료: 종이에 펜, 수채 물감

카테고리: 드로잉

작품 설명: 대향(大鄕) 이중섭(李仲燮, 1916-1956)은 평안북도 정주의 오산고등보통학교에서 서양화가 임용련, 백남순 부부에게 서양화를 배웠다. 이후 1936년 일본으로 건너가 데이코쿠미술학교(帝国美術学校)와 분카학원(文化学院)에서 미술을 전공했다. 추상 미술단체인 ‘자유미술가협회(自由美術家協会)’의 전시회에 지속적으로 출품하였으며, 제7회전(1943)에서는 태양상(太陽賞)을 수상했다. 1943년 귀국 후에는 생활고와 병으로 고생하면서도 꾸준히 작품을 제작했다.이중섭은 소, 아이들 등을 주요 소재로 고분 벽화와 민화 등 전통적이고 토속적인 것에 영감을 받아 표현주의적인 감각으로 작품을 제작했다. 이중섭의 작품에서는 그의 삶을 엿볼 수 있다. 동경의 분카학원에서 야마모토 마사코와 연애하던 시기의 엽서화에는 두 사람의 연인관계를 암시하는 환상적이고 초현실주의적인 이미지를 그렸다. 한국 전쟁기 제주도 피란시절 작품에는 가족과 행복했던 나날들이 소박하게 표현되었으며, 가족을 일본으로 보낸 후에는 삭막한 풍경화와 전쟁의 은유들이 그려졌다. 그는 열악한 경제 상황과 재료 부족에도 끊임없이 새로운 기법과 재료를 실험했는데, 담배를 싼 은지를 활용한 은지화가 대표적인 예이다. 전쟁이 끝난 후에는 가족을 만나려는 생각에 작품 제작에 몰두하여, 당당하고 힘찬 기세가 화면에 나타난다. 그러나 곧 경제적 어려움과 정신질환 등에 시달리며 가족과 재회할 수 있으리라는 희망이 사라졌을 때에는 초점을 잃은 흐릿한 풍경들이 애잔하게 펼쳐졌다.〈토끼풀〉은 이중섭이 이 시기 제작한 여러 점의 엽서화 중 하나로, 토끼풀 도상은 그의 그림에 반복적으로 등장하였다. 화면 중앙에 분리된 꽃과 잎을 큼지막한 도안처럼 연출하고 있고, 화면 좌우 사선으로 

In [42]:
# 검색 수행: 유사도 점수와 함께 반환
docs_and_scores = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 검색된 문서 수 출력
print(f"검색된 문서 수: {len(docs_and_scores)}")

# 각 문서의 파일명, 전체 내용, 유사도 점수 출력
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n문서 {i}:")
    print(f"  파일명: {doc.metadata.get('source', 'N/A')}")
    print(f"  유사도 점수: {score:.4f}")
    print(f"  전체 내용: {doc.page_content}")


검색된 문서 수: 5

문서 1:
  파일명: N/A
  유사도 점수: 0.2725
  전체 내용: 작품명: 토끼풀 / N/A / Clover

작가: 이중섭 / LEE Jungseop

작품 번호: 10001

제작 연도: 1941

크기: 9×14

재료: 종이에 펜, 수채 물감

카테고리: 드로잉

작품 설명: 대향(大鄕) 이중섭(李仲燮, 1916-1956)은 평안북도 정주의 오산고등보통학교에서 서양화가 임용련, 백남순 부부에게 서양화를 배웠다. 이후 1936년 일본으로 건너가 데이코쿠미술학교(帝国美術学校)와 분카학원(文化学院)에서 미술을 전공했다. 추상 미술단체인 ‘자유미술가협회(自由美術家協会)’의 전시회에 지속적으로 출품하였으며, 제7회전(1943)에서는 태양상(太陽賞)을 수상했다. 1943년 귀국 후에는 생활고와 병으로 고생하면서도 꾸준히 작품을 제작했다.이중섭은 소, 아이들 등을 주요 소재로 고분 벽화와 민화 등 전통적이고 토속적인 것에 영감을 받아 표현주의적인 감각으로 작품을 제작했다. 이중섭의 작품에서는 그의 삶을 엿볼 수 있다. 동경의 분카학원에서 야마모토 마사코와 연애하던 시기의 엽서화에는 두 사람의 연인관계를 암시하는 환상적이고 초현실주의적인 이미지를 그렸다. 한국 전쟁기 제주도 피란시절 작품에는 가족과 행복했던 나날들이 소박하게 표현되었으며, 가족을 일본으로 보낸 후에는 삭막한 풍경화와 전쟁의 은유들이 그려졌다. 그는 열악한 경제 상황과 재료 부족에도 끊임없이 새로운 기법과 재료를 실험했는데, 담배를 싼 은지를 활용한 은지화가 대표적인 예이다. 전쟁이 끝난 후에는 가족을 만나려는 생각에 작품 제작에 몰두하여, 당당하고 힘찬 기세가 화면에 나타난다. 그러나 곧 경제적 어려움과 정신질환 등에 시달리며 가족과 재회할 수 있으리라는 희망이 사라졌을 때에는 초점을 잃은 흐릿한 풍경들이 애잔하게 펼쳐졌다.〈토끼풀〉은 이중섭이 이 시기 제작한 여러 점의 엽서화 중 하나로, 토끼풀 도상은 그의 그림에 반복적으로 등장하였다. 화면 중앙에 분리된 꽃과 잎

In [24]:
import re

def extract_answer(text):
    # 정규 표현식을 사용하여 'Answer:' 이후의 모든 텍스트 추출
    match = re.search(r"Answer:\s*(.*)", text, re.DOTALL)
    if match:
        answer = match.group(1).strip()  # 불필요한 공백 제거
    else:
        answer = "답변을 찾을 수 없습니다."
    
    return answer

In [25]:
answer = extract_answer(response)
print(answer)

이중섭의 작품 중 두 가지를 추천드리겠습니다:

1. 《아버지와 두 아들》 (작품 번호: 9938):
   - 제작 연도: 1950
   - 크기: 30×41
   - 재료: 종이에 유화 물감
   - 작품 설명: 이 작품은 이중섭이 두 아들과의 즐겁던 추억을 그린 것입니다. 화면 상단에는 아버지가 한 아이의 하늘색 상의를 벗기려는 듯한 팔을 붙잡고 있고, 아이는 아버지의 무릎을 발로 밀어내며 이를 거부하는 듯 보입니다. 화면 하단에는 한 아이가 다리를 벌린 채 흥미진진하게 이 모습을 지켜보고 있습니다. 좌측 상단에는 검은 새 한 마리가 아이의 옷을 잡아당기고 있어 그림에 재미를 더합니다. 이중섭의 특유의 거친 필치와 굵은 윤곽선으로 표현된 인물들은 화면에 통일감을 주면서도 독특한 분위기를 자아냅니다.

2. 《두 사람》 (작품 번호: 10014):
   - 제작 연도: 1943


In [21]:
docs_and_scores[0][1]

0.212183877825737

In [72]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chroma 데이터베이스 불러오기 (임베딩 함수 포함)
persist_directory = './chroma_1104'
collection_name = 'chroma_art'

database = Chroma(
    embedding_function=embeddings,  # 임베딩 함수 설정
    collection_name=collection_name,
    persist_directory=persist_directory
)

# 모든 문서 검색을 위한 retriever 생성
retriever = database.as_retriever(search_kwargs={"k": 1000})

# 문서 검색 수행 (빈 쿼리 사용)
all_docs = retriever.get_relevant_documents(" ")

# 파일 소스 목록 출력
file_sources = set(doc.metadata['source'] for doc in all_docs)  # 중복 제거

print("데이터베이스에 저장된 파일 목록:")
for source in file_sources:
    print(source)


데이터베이스에 저장된 파일 목록:
./dataset2.docx
